In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.neighbors import KNeighborsRegressor

In [2]:
import pickle
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
np.random.RandomState(42)

import seaborn as sns

#import unittest
from nose.tools import *

import time
import datetime as dt
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm, skew, kurtosis

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV,cross_val_score 
from sklearn import preprocessing
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_log_error,mean_squared_error, make_scorer

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
import joblib
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

In [3]:
fin_ext_data = pd.read_csv("/Users/madhuranirale/Desktop/Project/Python_DS_Project/external_data_no_cyc.csv")

In [4]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [5]:
data1 = pd.read_parquet(Path("data") / "test.parquet")

In [6]:
fin_ext_data

,Unnamed: 0,date,temp,dwpt,rhum,prcp,wdir,wspd,pres,holiday,weekend,lockdown1,lockdown2,season,is_night
0,0,2020-01-01 00:00:00,0.8,-0.1,94,0.0,170.0,5.4,1032.5,1,0,0,0,3,1
1,1,2020-01-01 01:00:00,-0.7,-1.3,96,0.0,200.0,9.4,1032.2,1,0,0,0,3,1
2,2,2020-01-01 02:00:00,-0.3,-0.6,98,0.0,190.0,7.6,1032.2,1,0,0,0,3,1
3,3,2020-01-01 03:00:00,0.0,-0.3,98,0.0,220.0,5.4,1031.9,1,0,0,0,3,1
4,4,2020-01-01 04:00:00,0.3,0.0,98,0.0,210.0,7.6,1031.6,1,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17411,17411,2021-12-31 19:00:00,11.9,10.3,90,0.0,250.0,5.4,1026.5,0,0,0,0,3,0
17412,17412,2021-12-31 20:00:00,12.0,10.6,91,0.0,280.0,11.2,1026.6,0,0,0,0,3,1
17413,17413,2021-12-31 21:00:00,12.2,10.9,92,0.0,280.0,7.6,1026.6,0,0,0,0,3,1
17414,17414,2021-12-31 22:00:00,12.0,10.6,91,0.0,270.0,3.6,1026.7,0,0,0,0,3,1


In [7]:
fin_ext_data = fin_ext_data.drop(['Unnamed: 0'], axis=1)
fin_ext_data['date'] = pd.to_datetime(fin_ext_data['date'])

In [8]:
train_merged = data.merge(fin_ext_data, on='date')

In [9]:
test_merged = data1.merge(fin_ext_data, on='date')

In [10]:
data_tr = train_merged.sort_values(["date", "counter_name"])
y_train = data_tr["log_bike_count"].values
X_train = data_tr.drop(["log_bike_count", "bike_count"], axis=1)

In [11]:
data_ts = test_merged.sort_values(["date", "counter_name"])
y_test = data_ts["log_bike_count"].values
X_test = data_ts.drop(["log_bike_count", "bike_count"], axis=1)

In [12]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
import time

date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()
num_features = ['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres']

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name", "season"]

rest_cols = ['holiday', 'weekend', 'is_night', 'lockdown1', 'lockdown2']

preprocessor = ColumnTransformer(
    [
        ("date", OneHotEncoder(handle_unknown="ignore"), date_cols),
        ("cat", categorical_encoder, categorical_cols),
        ("numf", StandardScaler(), num_features),
        ("rem", 'passthrough', rest_cols)
    ]
)

params = {"lgbmregressor__learning_rate" : [0.01, 0.05, 0.1, 0.2],
          "lgbmregressor__max_depth" : [5, 6, 8, 9, 10, 12],
          "lgbmregressor__num_leaves" : [50, 100, 200, 400, 800, 1000],
          "lgbmregressor__min_data_in_leaf" : [50, 100, 200, 500],
          "lgbmregressor__lambda_l2": [1, 3, 5, 7, 9],
          "lgbmregressor__n_estimators" : [1000, 5000, 10000]
}

regressor = LGBMRegressor()
pipe = make_pipeline(date_encoder, preprocessor, regressor)
rscv = RandomizedSearchCV(pipe, param_distributions=params, n_iter=40, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=5, verbose=3)
rscv.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=9, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9
[CV 4/5] END lgbmregressor__lambda_l2=9, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=10, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=800;, score=-0.632 total time=27.8min
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=7, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7
[CV 3/5] END lgbmregressor__lambda_l2=7, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=8, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=5000, lgbmregressor__num_leaves=200;, score=-0.680 total time=13.1min
[LightGBM]

/Users/madhuranirale/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[CV 1/5] END lgbmregressor__lambda_l2=1, lgbmregressor__learning_rate=0.05, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=200;, score=-0.654 total time=32.8min
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=7, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7
[CV 5/5] END lgbmregressor__lambda_l2=7, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=8, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=5000, lgbmregressor__num_leaves=200;, score=-0.745 total time=11.6min
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 w

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[CV 3/5] END lgbmregressor__lambda_l2=5, lgbmregressor__learning_rate=0.2, lgbmregressor__max_depth=10, lgbmregressor__min_data_in_leaf=50, lgbmregressor__n_estimators=5000, lgbmregressor__num_leaves=1000;, score=-0.692 total time=15.1min
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] lambda_l2 is set=7, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7
[CV 5/5] END lgbmregressor__lambda_l2=7, lgbmregressor__learning_rate=0.1, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=50, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=50;, score=-0.734 total time=11.9min
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be 

[CV 3/5] END lgbmregressor__lambda_l2=3, lgbmregressor__learning_rate=0.05, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=200;, score=-0.667 total time= 1.9min
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[CV 5/5] END lgbmregressor__lambda_l2=3, lgbmregressor__learning_rate=0.05, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=200;, score=-0.729 total time= 1.7min
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] lambda_l2 is set=9, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9
[CV 3/5] END lgbmregressor__lambda_l2=9, lgbmregressor__learning_rate=0.1,

...

[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=9, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9
[CV 2/5] END lgbmregressor__lambda_l2=9, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=5, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=100;, score=-0.682 total time=  38.1s
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=9, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9
[CV 3/5] END lgbmregressor__lambda_l2=9, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=5, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=100;, score=-0.751 total time=  41.9s
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 wil

[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=9, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9
[CV 2/5] END lgbmregressor__lambda_l2=9, lgbmregressor__learning_rate=0.01, lgbmregressor__max_depth=10, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=800;, score=-0.637 total time=21.9min
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[CV 4/5] END lgbmregressor__lambda_l2=1, lgbmregressor__learning_rate=0.05, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=500, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=200;, score=-0.630 total time=29.7min
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20

[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[CV 5/5] END lgbmregressor__lambda_l2=5, lgbmregressor__learning_rate=0.1, lgbmregressor__max_depth=10, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=10000, lgbmregressor__num_leaves=200;, score=-0.719 total time=25.5min
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[CV 3/5] END lgbmregressor__lambda_l2=1, lgbmregressor__learning_rate=0.1, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=200, lgbmregressor__n_estimators=5000, lgbmregressor__num_leaves=100;, score=-0.668 total time=13.9min
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. 

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[CV 3/5] END lgbmregressor__lambda_l2=5, lgbmregressor__learning_rate=0.1, lgbmregressor__max_depth=9, lgbmregressor__min_data_in_leaf=100, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=100;, score=-0.674 total time= 1.3min
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[CV 2/5] END lgbmregressor__lambda_l2=3, lgbmregressor__learning_rate=0.2, lgbmregressor__max_depth=12, lgbmregressor__min_data_in_leaf=50, lgbmregressor__n_estimators=1000, lgbmregressor__num_leaves=100;, score=-0.638 total time= 1.9min
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('functiontransformer',
                                              FunctionTransformer(func=<function _encode_dates at 0x7f8fda7cb940>)),
                                             ('columntransformer',
                                              ColumnTransformer(transformers=[('date',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['year',
                                                                                'month',
                                                                                'day',
                                                                                'weekday',
                                                                                'hour']),
                                                                       

In [14]:
bst = rscv.best_params_
bst

{'lgbmregressor__num_leaves': 50,
 'lgbmregressor__n_estimators': 5000,
 'lgbmregressor__min_data_in_leaf': 500,
 'lgbmregressor__max_depth': 10,
 'lgbmregressor__learning_rate': 0.1,
 'lgbmregressor__lambda_l2': 7}

In [16]:
from sklearn.metrics import mean_squared_error, r2_score

print(
    f"Train set, RMSE={mean_squared_error(y_train, rscv.predict(X_train), squared=False):.7f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, rscv.predict(X_test), squared=False):.7f}"
)

print(
    f"Train set r2, RMSE={r2_score(y_train, rscv.predict(X_train)):.7f}"
)
print(
    f"Test set r2, RMSE={r2_score(y_test, rscv.predict(X_test)):.7f}"
)

Train set, RMSE=0.3420503
Test set, RMSE=0.4500828
Train set r2, RMSE=0.9583074
Test set r2, RMSE=0.9021840
